# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ResNet Model 

In [ ]:
import keras
import keras.utils
import time
from   keras import utils as np_utils
import tensorflow.compat.v1 as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.preprocessing import image
from keras.layers import GlobalAveragePooling2D, Dense, Dropout,Activation,Flatten
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D,MaxPooling2D,Flatten,Dense,Dropout,InputLayer
from tqdm import tqdm
import random
import datetime
import pandas as pd
from keras.models import load_model

def LoadData():
 
  counters = [0,0,0,0,0,0,0,0,0,0,0,0]
  path='/content/drive/My Drive/newdata/datasetall'
  X = []
  Y = []
  Names = []
  for image in tqdm(os.listdir(path)):
    image_path = os.path.join(path,image)
    img = cv2.imread(image_path)
    sizeNew = 224,224 
    resized = cv2.resize(img,sizeNew)
    if 'AN' in image :
     Y.append(0)
     counters[0] += 1
    elif 'CN' in image :
      Y.append(1)
      counters[1] += 1
    elif 'SN' in image :
      Y.append(2)
      counters[2] += 1
    elif 'AG' in image :
      Y.append(3)
      counters[3] += 1
    elif 'CG' in image :
      Y.append(4)
      counters[4] += 1
    elif 'SG' in image :
      Y.append(5)
      counters[5] += 1
    elif 'AM' in image :
      Y.append(6)
      counters[6] += 1
    elif 'CM' in image :
      Y.append(7)
      counters[7] += 1
    elif 'SM' in image :
      Y.append(8)
      counters[8] += 1
    elif 'AP' in image :
      Y.append(9)
      counters[9] += 1
    elif 'CP' in image :
      Y.append(10)
      counters[10] += 1
    elif 'SP' in image :
      Y.append(11)
      counters[11] += 1
    X.append(np.array(resized))  
    Names.append(image)
  return X,Y,Names    

def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result


def fill(img):
    img = cv2.resize(img, (224 , 224), cv2.INTER_CUBIC)
    return img


def zoom(img, value= 0.5):
    if value > 1 or value < 0:
        print('Value for zoom should be less than 1 and greater than 0')
        return img
    value = random.uniform(value, 1)
    h, w = img.shape[:2]
    h_taken = int(value*h)
    w_taken = int(value*w)
    h_start = random.randint(0, h-h_taken)
    w_start = random.randint(0, w-w_taken)
    img = img[h_start:h_start+h_taken, w_start:w_start+w_taken, :]
    img = fill(img)
    return img

def ResnetFunc (loopnum):
  Xx,Yx,Nn=LoadData()
  X = np.array(Xx)
  Y = np.array(Yx)
  N = np.array(Nn)

  X_train, X_Test, Y_train, Y_Test , N_train , N_test = train_test_split(X, Y,N,test_size=0.33, random_state=0,shuffle=True)

  
  X_train = np.array(X_train)
  Y_train = np.array(Y_train)

  X_Test = np.array(X_Test)
  Y_Test = np.array(Y_Test)
  
  N_test = np.array(N_test)
  
  input_shape = (224, 224, 3)
  
  
  iterator = 0 
  augmantedX = []
  augmantedY = []
  for g in Y_train :
    if g >= 6 :
      newimg = rotate_image(X_train[iterator],30)
      augmantedX.append(np.array(newimg))  
      augmantedY.append(g)
      if g == 11 :
        newimg = zoom(X_train[iterator],0.88)
        augmantedX.append(np.array(newimg))  
        augmantedY.append(g)
        newimg = rotate_image(newimg,330)
        augmantedX.append(np.array(newimg))  
        augmantedY.append(g)
        

    iterator+=1 
 
  s = np.concatenate((X_train ,augmantedX ), axis = 0)
  ss = np.concatenate((Y_train ,augmantedY ), axis = 0)
  X_train = s
  Y_train = ss
  X_train = np.array(X_train)
  Y_train = np.array(Y_train)
  
  ResnetModel = ResNet50(weights='imagenet',include_top=False,input_shape=input_shape)

  Last_layer=ResnetModel.layers[-1].output

  layer_output=keras.layers.Flatten()(Last_layer)
  
  ResnetModel=Model(ResnetModel.input,layer_output)
  
  model=Sequential()
 
  model.add(ResnetModel)
 
  model.add(Dropout(0.5))

  cl=Dense(12,activation='softmax')
  
  model.add(cl)

  d=keras.optimizers.SGD(lr=0.001)
  
  model.compile(loss='sparse_categorical_crossentropy',optimizer=d,metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  hist = model.fit(X_train, Y_train, batch_size=32, epochs=4, verbose=1, callbacks=[tensorboard_callback])
  
  (loss, accuracy) = model.evaluate(X_Test, Y_Test, batch_size=32, verbose=1)
  
  print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
  pathnew = '/content/drive/My Drive/lastrun'+str(loopnum)+'.csv'
  with open (pathnew , 'w') as f:
    f.write('name\n')
  with open (pathnew , 'a') as f:
    for u in N_test :
      f.write('{}\n'.format(u))
  return model 
def RwadTestData():
  path='/content/drive/My Drive/newdata/datasetall'
  pathtestonly = '/content/drive/My Drive/epochnumber2.csv'
  result = pd.read_csv(pathtestonly)
  data = result['name']
  X = []
  Y = []
  samples = []
  for image in tqdm(os.listdir(path)):
    if image in data.values :    
      image_path = os.path.join(path,image)
      img = cv2.imread(image_path)
      sizeNew = 224,224 
      resized = cv2.resize(img,sizeNew)
      if 'AN' in image :
        Y.append(0)
      elif 'CN' in image :
        Y.append(1)
      elif 'SN' in image :
        Y.append(2)
      elif 'AG' in image :
        Y.append(3)
      elif 'CG' in image :
        Y.append(4)
      elif 'SG' in image :
        Y.append(5)
      elif 'AM' in image :
        Y.append(6)
      elif 'CM' in image :
        Y.append(7)
      elif 'SM' in image :
        Y.append(8)
      elif 'AP' in image :
        Y.append(9)
      elif 'CP' in image :
        Y.append(10)
      elif 'SP' in image :
        Y.append(11)
      X.append(np.array(resized))  
  return X,Y   

'''
if os.path.exists('/content/drive/My Drive/Final_test _ata/BrainCancerModel2.h5') :
   model = keras.models.load_model('/content/drive/My Drive/Final_test _ata/BrainCancerModel2.h5')
   d=keras.optimizers.SGD(lr=0.001)
   model.compile(loss='sparse_categorical_crossentropy',optimizer=d,metrics=['accuracy'])
   model.load_weights('/content/drive/My Drive/Final_test _ata/BrainCancerModel2.h5') 
  
   x,y = RwadTestData()
   x=np.array(x)
   y=np.array(y)
   (loss, accuracy) = model.evaluate(x, y, batch_size=32, verbose=1)
  
   print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
 
   preds = model.predict(x)
   y_classes = preds.argmax(axis=-1)  
   temp = sum(y == y_classes)

   temp = temp/len(y)
   temp *=100
   print (temp)
    '''

for i in range(5):
  ResnetFunc(i)

100%|██████████| 4959/4959 [25:48<00:00,  3.20it/s]


94773248/94765736 [==============================] - 0s 0us/step
Epoch 1/4
139/139 [==============================] - 54s 309ms/step - loss: 1.2227 - accuracy: 0.7566
Epoch 2/4
139/139 [==============================] - 44s 316ms/step - loss: 0.0888 - accuracy: 0.9697
Epoch 3/4
139/139 [==============================] - 47s 335ms/step - loss: 0.0219 - accuracy: 0.9927
Epoch 4/4
52/52 [==============================] - 7s 112ms/step - loss: 0.0943 - accuracy: 0.9713
[INFO] loss=0.0943, accuracy: 97.1289%


100%|██████████| 4959/4959 [00:17<00:00, 279.56it/s]


Epoch 1/4
139/139 [==============================] - 54s 365ms/step - loss: 1.2130 - accuracy: 0.7633
Epoch 2/4
139/139 [==============================] - 49s 351ms/step - loss: 0.1052 - accuracy: 0.9680
Epoch 3/4
139/139 [==============================] - 50s 357ms/step - loss: 0.0427 - accuracy: 0.9848
Epoch 4/4
52/52 [==============================] - 7s 109ms/step - loss: 0.1093 - accuracy: 0.9676
[INFO] loss=0.1093, accuracy: 96.7624%


100%|██████████| 4959/4959 [00:17<00:00, 288.96it/s]


Epoch 1/4
139/139 [==============================] - 54s 369ms/step - loss: 1.1999 - accuracy: 0.7656
Epoch 2/4
139/139 [==============================] - 49s 352ms/step - loss: 0.0979 - accuracy: 0.9712
Epoch 3/4
139/139 [==============================] - 50s 357ms/step - loss: 0.0348 - accuracy: 0.9891
Epoch 4/4
52/52 [==============================] - 6s 108ms/step - loss: 0.1476 - accuracy: 0.9603
[INFO] loss=0.1476, accuracy: 96.0293%


100%|██████████| 4959/4959 [00:17<00:00, 290.97it/s]


Epoch 1/4
139/139 [==============================] - 54s 366ms/step - loss: 1.0769 - accuracy: 0.7730
Epoch 2/4
139/139 [==============================] - 49s 353ms/step - loss: 0.1096 - accuracy: 0.9668
Epoch 3/4
139/139 [==============================] - 50s 357ms/step - loss: 0.0413 - accuracy: 0.9863
Epoch 4/4
52/52 [==============================] - 6s 108ms/step - loss: 0.1239 - accuracy: 0.9664
[INFO] loss=0.1239, accuracy: 96.6402%


100%|██████████| 4959/4959 [00:17<00:00, 284.78it/s]


Epoch 1/4
139/139 [==============================] - 55s 366ms/step - loss: 1.4286 - accuracy: 0.7478
Epoch 2/4
139/139 [==============================] - 49s 352ms/step - loss: 0.1098 - accuracy: 0.9658
Epoch 3/4
139/139 [==============================] - 49s 355ms/step - loss: 0.0307 - accuracy: 0.9892
Epoch 4/4
52/52 [==============================] - 6s 107ms/step - loss: 0.2344 - accuracy: 0.9536
[INFO] loss=0.2344, accuracy: 95.3574%


In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \